In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from gaGen import GenGeneticAlgorithm
import copy

In [2]:
def evaluate(X_train : pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame, random_state = None):
    
    clf = DecisionTreeClassifier(random_state= random_state)
    
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    precision = precision_score(predictions,y_test, average='macro')
    f1 = f1_score(predictions,y_test, average='macro')
    recall = recall_score(predictions,y_test, average='macro')
    
    return {"precision":precision, "f1":f1, "recall":recall}

In [3]:
dataset_name= "balance-scale"

data = pd.read_csv(f"../Datasets/{dataset_name}.csv")

data.head(10)

,L-Weight,L-Distance,R-Weight,R-Distance,Class
0,1,1,1,1,B
1,1,1,1,2,R
2,1,1,1,3,R
3,1,1,1,4,R
4,1,1,1,5,R
5,1,1,2,1,R
6,1,1,2,2,R
7,1,1,2,3,R
8,1,1,2,4,R
9,1,1,2,5,R


In [4]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [5]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state= 42)

before_scores= []
selected_scores= []
all_scores = []
columns_number= []

for index,(train_index, test_index) in enumerate(skf.split(X, y)):
    
    fold_x_train =X.iloc[train_index]
    fold_y_train = y.iloc[train_index]
    
    fold_x_test =X.iloc[test_index]
    fold_y_test = y.iloc[test_index]
    
    score_before = evaluate(fold_x_train, fold_y_train, fold_x_test, fold_y_test)
    before_scores.append(score_before)
    
    print(f'''For the original data:
    Columns : {fold_x_train.columns}
    Columns Number : {len(fold_x_train.columns)} 
    precision : {score_before["precision"]}
    f1        : {score_before["f1"]}
    recall    : {score_before["recall"]}
          ''')
    print("Ga Gen started with inputs ......")
    ga= GenGeneticAlgorithm(
        X_train= fold_x_train,
        X_test= fold_x_test,
        y_train= fold_y_train,
        y_test= fold_y_test,
        rnd_state=None
    )

    selected_new_X_train , selected_new_X_test, all__new_X_train ,all_new_X_test = ga.run(population_size= 30, generation_limit= 50, mutation_probability= 0.1)
    
    print("Ga Gen finished ...")

    score_after= evaluate(all__new_X_train, fold_y_train, all_new_X_test, fold_y_test)
    all_scores.append(score_after)
    
    print(f'''\nFor the new generated data with gaGen without selection:
    Columns : {all_new_X_test.columns}
    Columns Number : {len(all_new_X_test.columns)} 
    precision : {score_after["precision"]}
    f1        : {score_after["f1"]}
    recall    : {score_after["recall"]}
    ''')
    score_after= evaluate(selected_new_X_train, fold_y_train, selected_new_X_test, fold_y_test)
    selected_scores.append(score_after)
    
    columns_number.append(len(selected_new_X_test.columns))
    
    print(f'''\nFor the new generated data with gaGen with selection:
    Columns : {selected_new_X_test.columns}
    Columns Number : {len(selected_new_X_test.columns)} 
    precision : {score_after["precision"]}
    f1        : {score_after["f1"]}
    recall    : {score_after["recall"]}
--------------------------------------------------------------------------------------''')
    

For the original data:
    Columns : Index(['L-Weight', 'L-Distance', 'R-Weight', 'R-Distance'], dtype='object')
    Columns Number : 4 
    precision : 0.5057471264367815
    f1        : 0.5333333333333333
    recall    : 0.5641025641025641
          
Ga Gen started with inputs ......
Ga Gen finished ...

For the new generated data with gaGen without selection:
    Columns : Index(['L-Weight', 'L-Distance', 'R-Weight', 'R-Distance',
       'L-Weight_L-Distance___addition', 'L-Weight_L-Distance___average',
       'L-Weight_L-Distance___multiplication', 'L-Weight_R-Weight___addition',
       'L-Weight_R-Weight___average', 'L-Weight_R-Weight___multiplication',
       'L-Weight_R-Distance___addition', 'L-Weight_R-Distance___average',
       'L-Weight_R-Distance___multiplication',
       'L-Distance_R-Weight___addition', 'L-Distance_R-Weight___average',
       'L-Distance_R-Weight___multiplication',
       'L-Distance_R-Distance___addition', 'L-Distance_R-Distance___average',
       'L-Dis

In [6]:
sums = {}
counts = {}

for score in before_scores:
    for key, value in score.items():
        sums[key] = sums.get(key, 0) + value
        counts[key] = counts.get(key, 0) + 1

means = {key: sums[key] / counts[key] for key in sums}

print("Original Data:")
for prop, mean in means.items():
    print(prop + ":", mean)

Original Data:
precision: 0.5659688013136288
f1: 0.5750237525757214
recall: 0.585870404022981


In [7]:
sums = {}
counts = {}

for score in all_scores:
    for key, value in score.items():
        sums[key] = sums.get(key, 0) + value
        counts[key] = counts.get(key, 0) + 1

means = {key: sums[key] / counts[key] for key in sums}

print("Generated without fs Data:")
for prop, mean in means.items():
    print(prop + ":", mean)

Generated without fs Data:
precision: 0.8782348111658458
f1: 0.8733324537160397
recall: 0.8753602398782228


In [8]:
sums = {}
counts = {}

for score in selected_scores:
    for key, value in score.items():
        sums[key] = sums.get(key, 0) + value
        counts[key] = counts.get(key, 0) + 1

means = {key: sums[key] / counts[key] for key in sums}

print("Generated with fs Data:")
for prop, mean in means.items():
    print(prop + ":", mean)

Generated with fs Data:
precision: 0.9520689655172415
f1: 0.9566300638602157
recall: 0.9660651164433146


In [9]:
print(f"Columns generation: {np.mean(columns_number)}")

Columns generation: 4.2
